# Преобразобавние данных с помощью RunnableParallel

## Преобразование входных и выходных данных

Интерфейс RunnableParallel может быть полезен для преобразования выходных данных одного Runnable-интерфейса, в формат входных данных следующего Runnable-интерфейса в последовательности.

В примере ниже входные данные для объейта `prompt` должны быть представлены в виде map-структуры с ключами `context` и `question`.
При этом, данные введенные пользователем, представляют обычный вопрос.
Эти данные передаются в поле `question`, а для получения контекста и заполнения поля `context` используется ретривер.

In [ ]:
%pip install --upgrade --quiet  gigachain gigachat

In [79]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

vectorstore = FAISS.from_texts(
    ["Василий работал в Сбербанке"], embedding=GigaChatEmbeddings(credentials="<авторизационные_данные>", verify_ssl_certs=False)
)
retriever = vectorstore.as_retriever()
template = """Отвечай на вопрос только на основе контекста:
{context}

Вопрос: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("Где работал Василий?")

'Василий работал в Сбербанке.'

:::note

Типы преобразуются автоматически, поэтому при соединений интерфейсов RunnableParallel и Runnable не нужно оборачивать словарь в экземпляр класса RunnableParallel.

Таким образом, в контексте цепочки оба следующих способа будут работать одинаково:

:::

```json
{"context": retriever, "question": RunnablePassthrough()}
```

```python
RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
```

```python
RunnableParallel(context=retriever, question=RunnablePassthrough())
```



## Сокращение кода с помощью itemgetter

При работе с `RunnableParallel` вы можете использовать Python-функцию [`itemgetter`](https://docs.python.org/3/library/operator.html#operator.itemgetter), чтобы сократить код, который нужен для извлечения данных из map-структуры.

Приведенный пример показывает как использовать `itemgetter` для получения определенных ключей:

In [77]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

vectorstore = FAISS.from_texts(
    ["Василий работал в Сбербанке"], embedding=GigaChatEmbeddings(credentials="<авторизационные_данные>", verify_ssl_certs=False)
)
retriever = vectorstore.as_retriever()

template = """Отвечай на вопрос только на основе контекста:
{context}

Вопрос: {question}

Переведи ответ на заданный язык: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "где работал Вася", "language": "английский"})

'Ответ: Vasyа worked at Sberbank.'

## Распараллеливание этапов работы

RunnableParallel (или RunnableMap) позволяет параллельно выполнять несколько Runnable-интерфейсов и возвращать их выходные данные в виде map-структуры.

In [80]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain.chat_models.gigachat import GigaChat

model = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)
joke_chain = ChatPromptTemplate.from_template("расскажи шутку про {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("напиши короткий стих про {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "облако"})

{'joke': AIMessage(content='Почему облака не любят понедельники? Потому что они всегда идут после субботы и воскресенья.', response_metadata={'token_usage': Usage(prompt_tokens=18, completion_tokens=24, total_tokens=42), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'}),
 'poem': AIMessage(content='Облако плывет по небу,\nСловно белый парус в синеве.\nПусть оно не знает грусти,\nПусть оно всегда на высоте!', response_metadata={'token_usage': Usage(prompt_tokens=19, completion_tokens=40, total_tokens=59), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'})}

## Параллелизм

Так как каждый Runnable-интерфейс в map-структуре исполняется параллельно, интерфейс RunnableParallel можно использовать для параллельного выполнения независимых процессов.
Так, приведенные выше цепочки  `joke_chain`, `poem_chain` и `map_chain` выполняются примерно одинаковое время, при том, что `map_chain` включает выполнение двух других цепочек.

In [83]:
%%timeit

joke_chain.invoke({"topic": "облако"})

1.01 s ± 385 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
%%timeit

poem_chain.invoke({"topic": "облако"})

9.7 s ± 219 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%%timeit

map_chain.invoke({"topic": "облако"})

9.73 s ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
